In [1]:
library(ArchR)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
getwd()

[1] "/project/scATAC_analysis/scATAcat_review/preprocessing/Corces2020_brain_scATAC/04_get_peaks_and_genesScore_matrix_ArchR"

In [3]:
set.seed(2022)
addArchRThreads(threads = 1) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 1.

Setting default genome to Hg38.



In [7]:
inputFiles = "/project/scATAC_analysis/10X_scATAC_HumanBrain_Corces2020/analysis/get_ENCODE_cCRE_by_cell_coverage/outs/human_brain_Corces2020_atac_fragments_only_stdChr.tsv.gz"

In [8]:
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = "Corces2020_brain",
  minTSS = 4, #Dont set this too high because you can always increase later
  minFrags = 1000, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-18ed61112e72e-Date-2024-05-27_Time-18-22-13.555721.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2024-05-27 18:22:16.550348 : Batch Execution w/ safelapply!, 0 mins elapsed.

(Corces2020_brain : 1 of 1) Determining Arrow Method to use!

Attempting to index /project/scATAC_analysis/10X_scATAC_HumanBrain_Corces2020/analysis/get_ENCODE_cCRE_by_cell_coverage/outs/human_brain_Corces2020_atac_fragments_only_stdChr.tsv.gz as tabix..

2024-05-27 18:23:08.374785 : (Corces2020_brain : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.864 mins elapsed.

2024-05-27 18:23:08.487443 : (Corces2020_brain : 1 of 1) Tabix Bed To Temporary File, 0.866 mins elapsed.

Warning message in sprintf("%s Reading TabixFile %s Percent", prefix, round(100 * :
“one argument not used by format '%s Reading T

In [9]:
proj1 <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = "Corces2020_brain",
  copyArrows = TRUE #This is recommened so that you maintain an unaltered copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [10]:
getAvailableMatrices(proj1)

[1] "GeneScoreMatrix" "TileMatrix"

In [11]:
out_path = "../../../data/Corces2020_brain_scATAC/04_peaks_and_genesScore_matrix_ArchR/input_for_EpiAnno/" # where to save resulting matrices | with / at the end
threads_=1 # how many threads to use

proj1 <- addIterativeLSI(
    ArchRProj = proj1,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)

proj1 <- addClusters(
    input = proj1,
    reducedDims = "IterativeLSI",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.8
)

proj1 <- addUMAP(
    ArchRProj = proj1, 
    reducedDims = "IterativeLSI", 
    name = "UMAP", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine"
)

HDF5_USE_FILE_LOCKING=FALSE
RHDF5_USE_FILE_LOCKING=FALSE

proj2 <- addGroupCoverages(ArchRProj = proj1, groupBy = "Clusters")

pathToMacs2 <- findMacs2()

proj2 <- addReproduciblePeakSet(
    ArchRProj = proj2, 
    groupBy = "Clusters", 
    pathToMacs2 = pathToMacs2,
    plot = FALSE
)



Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-18ed6680a37bd-Date-2024-05-27_Time-19-05-19.489029.log
If there is an issue, please report to github with logFile!

2024-05-27 19:05:22.471525 : Computing Total Across All Features, 0.001 mins elapsed.

2024-05-27 19:05:25.371045 : Computing Top Features, 0.049 mins elapsed.

###########
2024-05-27 19:05:27.212371 : Running LSI (1 of 2) on Top Features, 0.08 mins elapsed.
###########

2024-05-27 19:05:27.363389 : Sampling Cells (N = 10000) for Estimated LSI, 0.082 mins elapsed.

2024-05-27 19:05:27.412642 : Creating Sampled Partial Matrix, 0.083 mins elapsed.

2024-05-27 19:05:58.886744 : Computing Estimated LSI (projectAll = FALSE), 0.608 mins elapsed.

2024-05-27 19:07:50.514752 : Identifying Clusters, 2.468 mins elapsed.

2024-05-27 19:08:20.849721 : Identified 7 Clusters, 2.974 mins elapsed.

2024-05-27 19:08:21.05506 : Saving LSI Iteration, 2.977 mins elapsed.

Found more than one class "dist" in cache; using t


************************************************************
2024-05-27 19:08:50.814721 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-18ed6680a37bd-Date-2024-05-27_Time-19-05-19.489029.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2024-05-27 19:08:50.821408 : Creating Cluster Matrix on the total Group Features, 3.473 mins elapsed.

2024-05-27 19:10:06.213235 : Computing Variable Features, 4.73 mins elapsed.

###########
2024-05-27 19:10:06.539005 : Running LSI (2 of 2) on Variable Features, 4.735 mins elapsed.
###########

2024-05-27 19:10:06.640842 : Creating Partial Matrix, 4.737 mins elapsed.

2024-05-27 19:10:40.457752 : Computing LSI, 5.301 mins elapsed.

2024-05-27 19:12:08.40527 : Finished Running IterativeLSI, 6.766 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-addClusters-18ed64e22c52a-Date-2024-05-27_Time-19-12-08.521957.log
If there is an issue, please report to github with logFile!

2024-05-27 19:12:11.890805 : Running Seurats FindClusters (Stuart et al. Cell 2019), 0.003 mins elapsed.

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 11941
Number of edges: 504895

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8829
Number of communities: 16
Elapsed time: 2 seconds


1 singletons identified. 15 final clusters.

2024-05-27 19:12:29.76863 : Testing Biased Clusters, 0.301 mins elapsed.

2024-05-27 19:12:29.948263 : Testing Outlier Clusters, 0.304 mins elapsed.

2024-05-27 19:12:30.01018 : Assigning Cluster Names to 15 Clusters, 0.305 mins elapsed.

2024-05-27 19:12:30.156479 : Finished addClusters, 0.307 mins elapsed.

19:12:30 UMAP embedding parameters a = 0.583 b = 1.334

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

19:12:30 Read 11941 rows and found 30 numeric columns

19:12:30 Using Annoy for neighbor search, n_neighbors = 30

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

19:12:30 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


...

Annotating Peaks : Nearest Gene

Annotating Peaks : Gene

Annotating Peaks : TSS

Annotating Peaks : GC



[1] "/project/scATAC_analysis/scATAcat_review/preprocessing/Corces2020_brain_scATAC/04_get_peaks_and_genesScore_matrix_ArchR/Corces2020_brain/PeakCalls/C8-reproduciblePeaks.gr.rds"


Annotating Peaks : Nearest Gene

Annotating Peaks : Gene

Annotating Peaks : TSS

Annotating Peaks : GC



[1] "/project/scATAC_analysis/scATAcat_review/preprocessing/Corces2020_brain_scATAC/04_get_peaks_and_genesScore_matrix_ArchR/Corces2020_brain/PeakCalls/C9-reproduciblePeaks.gr.rds"


2024-05-27 20:49:21.126271 : Creating Union Peak Set!, 41.525 mins elapsed.

Converged after 7 iterations!

2024-05-27 20:49:29.485389 : Finished Creating Union Peak Set (224481)!, 41.664 mins elapsed.



In [12]:
proj3 <- addPeakMatrix(proj2)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-18ed612bd9d88-Date-2024-05-27_Time-20-50-54.15089.log
If there is an issue, please report to github with logFile!

2024-05-27 20:50:54.993906 : Batch Execution w/ safelapply!, 0 mins elapsed.

2024-05-27 20:50:57.012919 : Adding Corces2020_brain to PeakMatrix for Chr (1 of 23)!, 0.028 mins elapsed.

2024-05-27 20:51:14.600883 : Adding Corces2020_brain to PeakMatrix for Chr (2 of 23)!, 0.321 mins elapsed.

2024-05-27 20:51:25.35719 : Adding Corces2020_brain to PeakMatrix for Chr (3 of 23)!, 0.501 mins elapsed.

2024-05-27 20:51:35.00565 : Adding Corces2020_brain to PeakMatrix for Chr (4 of 23)!, 0.661 mins elapsed.

2024-05-27 20:51:42.742808 : Adding Corces2020_brain to PeakMatrix for Chr (5 of 23)!, 0.79 mins elapsed.

2024-05-27 20:51:51.036091 : Adding Corces2020_brain to PeakMatrix for Chr (6 of 23)!, 0.929 mins elapsed.

2024-05-27 20:52:00.827502 : Adding Corces2020_brain to PeakMatrix for Chr (7 of 23)!, 1.092 mins elapsed.

2024-

In [13]:
peakMatrix_ = getMatrixFromProject(proj3, useMatrix='PeakMatrix')

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-18ed65ded4514-Date-2024-05-27_Time-20-53-55.855521.log
If there is an issue, please report to github with logFile!

2024-05-27 20:54:37.672151 : Organizing colData, 0.697 mins elapsed.

2024-05-27 20:54:37.698124 : Organizing rowData, 0.697 mins elapsed.

2024-05-27 20:54:37.728092 : Organizing rowRanges, 0.698 mins elapsed.

2024-05-27 20:54:37.770519 : Organizing Assays (1 of 1), 0.699 mins elapsed.

2024-05-27 20:54:37.825795 : Constructing SummarizedExperiment, 0.7 mins elapsed.

2024-05-27 20:54:43.768018 : Finished Matrix Creation, 0.799 mins elapsed.



In [ ]:

PeakMatrix_proj3_dense =  as.matrix(peakMatrix_@assays@data$PeakMatrix)

write.csv(PeakMatrix_proj3_dense, file = paste(out_path,"PeakMatrix_dense.csv",sep="" )) 

# get the peak regions
proj3_peak_metadata  = (getPeakSet(proj3))
proj3_peak_regions = data.frame(seqnames = seqnames(proj3_peak_metadata),
           start= start(proj3_peak_metadata),
            end = end(proj3_peak_metadata))

write.csv(proj3_peak_metadata, file =paste(out_path,"peak_metadata.csv",sep="" ))
write.csv(proj3_peak_regions, file =paste(out_path,"peak_regions.csv",sep="" ))
write.table(proj3_peak_regions, file =paste(out_path,"peak_regions.tsv",sep="" ),col.names = F,row.names=FALSE,sep="\t", quote = FALSE)

# save the peak matrix as sparse mtx


# save sparse matrix
sparse_matrix <- Matrix(PeakMatrix_proj3_dense , sparse = T )
writeMM(obj = sparse_matrix, file = paste(out_path,"ArchR_peakMatrix.mtx",sep="" ))

# save genes and cells names
combined_peak_features =  paste(proj3_peak_regions$seqnames,"_",proj3_peak_regions$start,"_",proj3_peak_regions$end, sep="")

write.table(as.data.frame(combined_peak_features), file = paste(out_path,"ArchR_peaks.tsv",sep="" ) , row.names = FALSE, quote = FALSE, col.names = FALSE)


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 20.0 GiB”


In [28]:
getAvailableMatrices(proj3)

[1] "GeneScoreMatrix" "PeakMatrix"      "TileMatrix"

In [ ]:
# get the gene activity matrix 

geneScoreMatrix <- getMatrixFromProject(proj3, useMatrix='GeneScoreMatrix')




In [ ]:
dim(geneScoreMatrix)

In [ ]:
geneScore_outDir = "../../../data/Corces2020_brain_scATAC/04_peaks_and_genesScore_matrix_ArchR/"

In [ ]:
write.table(rowData(geneScoreMatrix)$name, paste(geneScore_outDir, "ArchR_genescore_genes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [ ]:
write.table(colnames(geneScoreMatrix), paste(geneScore_outDir, "ArchR_genescore_barcodes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [ ]:
geneScoreMatrix_sparse = geneScoreMatrix@assays@data@listData$GeneScoreMatrix


In [ ]:
writeMM(obj = geneScoreMatrix_sparse, file=paste(geneScore_outDir, "ArchR_genescore.mtx", sep=""))

In [ ]:
saveArchRProject(ArchRProj = proj3, outputDirectory = paste(geneScore_outDir,"Save-Proj", sep=""), load = FALSE)

In [ ]:
library(stringr)

write.table(as.data.frame(str_split_fixed(colnames(geneScoreMatrix),'#',2))$V2, paste(geneScore_outDir, "cell_IDs.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)
